In [1]:
import os 
import pandas as pd
    
import json
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import allel
import itertools
from matplotlib import gridspec
import matplotlib.pyplot as plt
import pickle
from statannot import add_stat_annotation
from scipy import stats
import matplotlib.cm as cm
import matplotlib

from math import pi
import scipy

import matplotlib.patches as mpatches
import statsmodels.stats as sts_m
import mne
import statsmodels.api as sm
from tqdm import tqdm

In [2]:
pd.options.display.max_columns=200
pd.options.display.max_rows=100
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [3]:
df_met = pd.read_csv("../results/data/processed_hmf_escape_info.tsv.gz",sep="\t")
df_primary = pd.read_csv("../results/data//processed_pcawg_escape_info.tsv.gz",sep="\t")
df_meta = pd.read_csv("../metadata/dataset_metadata_supp_table3.tsv",sep="\t")
df_comb=pd.concat([df_primary,df_met])

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (16,17,19,20,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Select only cancer types with >=15 samples

In [6]:
x=df_comb["cancer_type_code"].value_counts()
ttypes_selected=x[x>15].index

In [7]:
def perform_fisher_exact_test(df_comb,column_query,column_dependent,threshold_min=0):
    l=[]
    for ttype,v in df_comb[(df_comb[column_query])&(df_comb["cancer_type_code"].isin(ttypes_selected))]["cancer_type_code"].value_counts().items():
        if v >= threshold_min:
            
            samples_t = list(df_comb[(df_comb[column_query])&(df_comb["cancer_type_code"]==ttype)]["sample_id"].values)
            samples_nt = list(df_comb[(~df_comb[column_query])&(df_comb["cancer_type_code"]==ttype)]["sample_id"].values)
            # enrichment
            a=df_comb[(df_comb["sample_id"].isin(samples_t))&(df_comb[column_dependent])].shape[0]
            b=df_comb[(df_comb["sample_id"].isin(samples_t))&(~df_comb[column_dependent])].shape[0]
            
            c=df_comb[(df_comb["sample_id"].isin(samples_nt))&(df_comb[column_dependent])].shape[0]
            d=df_comb[(df_comb["sample_id"].isin(samples_nt))&(~df_comb[column_dependent])].shape[0]
            o,p=scipy.stats.fisher_exact([[a,b],[c,d]],alternative="two-sided")
            if np.isfinite(p):
                l.append([ttype,len(samples_t),len(samples_nt),a,b,c,d,np.log2(o),p])
    df_t =pd.DataFrame(l,columns=["ttype","n_samples_group","n_samples_no_group","a","b","c","d","odds","pvalue"]).drop_duplicates()
    df_t["qvalue"]=mne.stats.fdr_correction(df_t["pvalue"])[1]
    df_t["column"] = column_query
    df_t["variable_dependent"] = column_dependent
    return df_t



### Load dependent variables

In [8]:
df_d = pd.read_csv("../results/data/features_correlation/randomized_escape_for_features.tsv.gz",sep="\t")
df_d1 = pd.read_csv("../results/data/features_correlation/escape_for_features.tsv",sep="\t")

# HLA-I supertypes

In [13]:
df_sts = pd.read_csv("../results/data/features_correlation/supertypes_per_sample.tsv",sep="\t").merge(df_d).merge(df_d1)
supertypes=[st for st in df_sts.columns.values if "_st" in st and st[0] in ["A","B","C"]  ]
l,l1=[],[]
for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
    for st in supertypes:
        l.append(perform_fisher_exact_test(df_sts,st,x,threshold_min=50))
df_sts_discrete=pd.concat(l)
for st in supertypes:
    for c in list(df_d.columns.values[101:301]):
        l1.append(perform_fisher_exact_test(df_sts,st,c,threshold_min=50))
df_sts_discrete_control=pd.concat(l1)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [14]:
df_sts_discrete.to_csv("../results/data/results_feature_correlation/supertypes_analysis_results.tsv",sep="\t",index=False)
df_sts_discrete_control.to_csv("../results/data/results_feature_correlation/supertypes_analysis_results_control.tsv",sep="\t",index=False)

# Viral DNA insertion

In [15]:
df_viral=pd.read_csv("../results/data/features_correlation/sample_specific_info_viral_integration.tsv",sep="\t").merge(df_d1).merge(df_d)
l=[]
for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
    l.append(perform_fisher_exact_test(df_viral,"viral_insertion",x,threshold_min=5))
df_viral_test = pd.concat(l)
l1=[]
for c in list(df_d.columns.values[101:301]):
    l1.append(perform_fisher_exact_test(df_viral,"viral_insertion",c,threshold_min=5))
df_viral_test_control=pd.concat(l1)



/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [16]:
df_viral_test.to_csv("../results/data/results_feature_correlation/viral_analysis_results.tsv",sep="\t",index=False)
df_viral_test_control.to_csv("../results/data/results_feature_correlation/viral_analysis_results_control.tsv",sep="\t",index=False)

# MSI

In [17]:
df_msi=df_comb[["sample_id","msi_status","cancer_type_code"]].merge(df_d1).merge(df_d)
df_msi["is_msi"] = df_msi["msi_status"] == "MSI"
l=[]
for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
    l.append(perform_fisher_exact_test(df_msi,"is_msi",x,threshold_min=5))
df_msi_discrete=pd.concat(l)

In [18]:
l1=[]
for c in list(df_d.columns.values[101:301]):
    l1.append(perform_fisher_exact_test(df_msi,"is_msi",c,threshold_min=5))
df_msi_discrete_control=pd.concat(l1)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [19]:
df_msi_discrete.to_csv("../results/data/results_feature_correlation/msi_analysis_results.tsv",sep="\t",index=False)
df_msi_discrete_control.to_csv("../results/data/results_feature_correlation/msi_analysis_results_control.tsv",sep="\t",index=False)

# HRD

In [20]:
df_hrd=df_comb[["sample_id","hr_status","cancer_type_code"]].merge(df_d1).merge(df_d)
df_hrd["is_hrd"] = df_hrd["hr_status"] == "HR_deficient"
print (df_msi.shape[0])
l=[]
for x in ["selected_genetic_immune_escape","excluding_loh_hla",]:
    l.append(perform_fisher_exact_test(df_hrd,"is_hrd",x,threshold_min=5))
df_hrd_discrete=pd.concat(l)

6319


/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [21]:
l1=[]
for c in list(df_d.columns.values[101:301]):
    l1.append(perform_fisher_exact_test(df_hrd,"is_hrd",c,threshold_min=5))
df_hrd_discrete_control=pd.concat(l1)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [22]:
df_hrd_discrete.to_csv("../results/data/results_feature_correlation/hrd_analysis_results.tsv",sep="\t",index=False)
df_hrd_discrete_control.to_csv("../results/data/results_feature_correlation/hrd_analysis_results_control.tsv",sep="\t",index=False)

# Whole-genome duplication

In [23]:
df_wgd=df_comb[["sample_id","whole_genome_duplication","cancer_type_code"]].merge(df_d1).merge(df_d)
l=[]
for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
    l.append(perform_fisher_exact_test(df_wgd,"whole_genome_duplication",x,threshold_min=5))
df_wgd_discrete=pd.concat(l)



/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [24]:
l1=[]
for c in list(df_d.columns.values[101:301]):
    l1.append(perform_fisher_exact_test(df_wgd,"whole_genome_duplication",c,threshold_min=5))
df_wgd_discrete_control=pd.concat(l1)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [25]:
df_wgd_discrete.to_csv("../results/data/results_feature_correlation/wgd_analysis_results.tsv",sep="\t",index=False)
df_wgd_discrete_control.to_csv("../results/data/results_feature_correlation/wgd_analysis_results_control.tsv",sep="\t",index=False)

### Pre-treatment

In [26]:
df_comb[(df_comb["cohort"]=="Hartwig")&(pd.isna(df_comb["had_chemotherapy"]))].shape

(2212, 153)

In [27]:
df_pret=df_comb[(df_comb["cohort"]=="Hartwig")&(~pd.isna(df_comb["had_chemotherapy"]))][["sample_id","had_chemotherapy","had_hormone_therapy","had_immunotherapy","had_targeted_therapy","cancer_type_code"]].merge(df_d1).merge(df_d).fillna(False)
l=[]
for c in ["had_chemotherapy","had_hormone_therapy","had_immunotherapy","had_targeted_therapy"]:
    for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
        l.append(perform_fisher_exact_test(df_pret,c,x,threshold_min=5))
df_pret=df_comb[(df_comb["cohort"]=="Hartwig")&(~pd.isna(df_comb["had_radiotherapy"]))][["sample_id","had_radiotherapy","cancer_type_code"]].merge(df_d1).merge(df_d).fillna(False)

for c in ["had_radiotherapy"]:
    for x in ["selected_genetic_immune_escape","excluding_loh_hla"]:
        l.append(perform_fisher_exact_test(df_pret,c,x,threshold_min=5))
        

df_pret_discrete = pd.concat(l)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [39]:
df_pret=df_comb[(df_comb["cohort"]=="Hartwig")&(~pd.isna(df_comb["had_chemotherapy"]))][["sample_id","had_chemotherapy","had_hormone_therapy","had_immunotherapy","had_targeted_therapy","cancer_type_code"]].merge(df_d1).merge(df_d).fillna(False)
l1=[]
for c in ["had_chemotherapy","had_hormone_therapy","had_immunotherapy","had_targeted_therapy"]:
    for x in list(df_d.columns.values[101:301]):
        l1.append(perform_fisher_exact_test(df_pret,c,x,threshold_min=5))
df_pret=df_comb[(df_comb["cohort"]=="Hartwig")&(~pd.isna(df_comb["had_radiotherapy"]))][["sample_id","had_radiotherapy","cancer_type_code"]].merge(df_d1).merge(df_d).fillna(False)

for c in ["had_radiotherapy"]:
    for x in list(df_d.columns.values[101:301]):
        l1.append(perform_fisher_exact_test(df_pret,c,x,threshold_min=5))
        

df_pret_discrete_control = pd.concat(l1)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()


In [40]:
df_pret_discrete.to_csv("../results/data/results_feature_correlation/pretreatment_analysis_results.tsv",sep="\t",index=False)
df_pret_discrete_control.to_csv("../results/data/results_feature_correlation/pretreatment_analysis_results_control.tsv",sep="\t",index=False)